<a href="https://colab.research.google.com/github/Chiamakac/TRAININGS/blob/main/TRAININGS/IgboNER-Models%20/Confusion%20matrix/Copy_of_Tuned_Distil_with_merged_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Distil Token classification (PyTorch)

Install the Transformers and Datasets libraries to run this notebook.

In [ ]:
!pip install datasets transformers[sentencepiece]
!pip install evaluate
#pip install --upgrade datasets
#!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.3 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


You will need to setup git, adapt your email and name in the following cell.

In [ ]:
!git config --global user.email "chukwunekechiamaka3@gmail.com"
!git config --global user.name "chiamaka"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset('masakhaner', 'ibo')

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2235
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 320
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 638
    })
})

In [ ]:
raw_datasets["train"][0]["tokens"]

['Ike',
 'ịda',
 'jụụ',
 'otụ',
 'nkeji',
 'banyere',
 'oke',
 'ogbugbu',
 'na',
 '-',
 'eme',
 "n'ala",
 'Naijiria',
 'agwụla',
 'Ekweremmadụ']

In [ ]:
raw_datasets["train"][0]["ner_tags"]

[0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 5, 0, 1]

In [ ]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-DATE', 'I-DATE'], id=None), length=-1, id=None)

In [ ]:
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-DATE', 'I-DATE']

In [ ]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

Ike ịda jụụ otụ    nkeji  banyere oke ogbugbu na - eme n'ala Naijiria agwụla Ekweremmadụ 
O   O   O   B-DATE I-DATE O       O   O       O  O O   O     B-LOC    O      B-PER       


In [ ]:
from transformers import AutoTokenizer

model_checkpoint ="distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
tokenizer.is_fast

True

In [ ]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'ike',
 'ida',
 'ju',
 '##u',
 'ot',
 '##u',
 'nk',
 '##ej',
 '##i',
 'ban',
 '##yer',
 '##e',
 'ok',
 '##e',
 'og',
 '##bu',
 '##gb',
 '##u',
 'na',
 '-',
 'em',
 '##e',
 'n',
 "'",
 'ala',
 'na',
 '##iji',
 '##ria',
 'ag',
 '##wu',
 '##la',
 'ek',
 '##wer',
 '##em',
 '##mad',
 '##u',
 '[SEP]']

In [ ]:
inputs.word_ids()

[None,
 0,
 1,
 2,
 2,
 3,
 3,
 4,
 4,
 4,
 5,
 5,
 5,
 6,
 6,
 7,
 7,
 7,
 7,
 8,
 9,
 10,
 10,
 11,
 11,
 11,
 12,
 12,
 12,
 13,
 13,
 13,
 14,
 14,
 14,
 14,
 14,
 None]

In [ ]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [ ]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 5, 0, 1]
[-100, 0, 0, 0, 0, 7, 8, 8, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 6, 0, 0, 0, 1, 2, 2, 2, 2, -100]


In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/638 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,    0,    0,    0,    0,    7,    8,    8,    8,    8,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    5,    6,    6,    0,    0,    0,    1,    2,    2,    2,
            2, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100],
        [-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    5,    6,    6,    0,    1,    2,    2,    2,    2,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    7,    8,    8,    8,    8,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    5,    6,    6, 

In [ ]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 0, 0, 0, 0, 7, 8, 8, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 6, 0, 0, 0, 1, 2, 2, 2, 2, -100]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 6, 0, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 6, 0, 0, 0, 0, 0, 0, -100]


In [ ]:
!pip install seqeval

In [ ]:
#from datasets import load_metric

#metric = load_metric("seqeval")

from evaluate import load

metric = load("seqeval")

In [ ]:
labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['O',
 'O',
 'O',
 'B-DATE',
 'I-DATE',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LOC',
 'O',
 'B-PER']

In [ ]:
predictions = labels.copy()
predictions[2] = "O"
metric.compute(predictions=[predictions], references=[labels])

{'DATE': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'LOC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'PER': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [ ]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [ ]:
id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.config.num_labels

9

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    "distilbert-base-uncased-nerR",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=20,
    weight_decay=0.01,
    #push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

<ipython-input-105-ddd9b7f8600f>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.415466,0.850829,0.770000,0.808399,0.958688
2,0.000700,0.329752,0.825397,0.780000,0.802057,0.961529
3,0.000700,0.337470,0.790000,0.790000,0.790000,0.963071
4,0.000700,0.389927,0.831826,0.766667,0.797918,0.959744
5,0.000700,0.335525,0.832753,0.796667,0.814310,0.963396
6,0.001000,0.375345,0.814035,0.773333,0.793162,0.960880
7,0.001000,0.345613,0.819788,0.773333,0.795883,0.960961
8,0.000800,0.370982,0.812721,0.766667,0.789022,0.959338
9,0.000900,0.342245,0.815009,0.778333,0.796249,0.960555
10,0.000900,0.339391,0.819930,0.781667,0.800341,0.960636


TrainOutput(global_step=5600, training_loss=0.000605285566832338, metrics={'train_runtime': 576.4977, 'train_samples_per_second': 77.537, 'train_steps_per_second': 9.714, 'total_flos': 827507073724002.0, 'train_loss': 0.000605285566832338, 'epoch': 20.0})

In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "/content/distilbert-base-uncased-nerR/checkpoint-5600"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("Google chetara ụbọchị ọmụmụ Keshi mgbe Obasanjo chịrị Naịjirịa afọ asatọ na ọchịchị onye kwuo uche ya."
                 "Onyeisi ndị na-emenyu ọkụ na, Legọọsi steeti bụ Rasak Fadipe ekwuola na onweghi onye nwụrụ n'ime ọkụ ahụ gbara n'ehihie ụbọchị 24 Jenuwarị, 2018.")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'entity_group': 'ORG',
  'score': 0.99983144,
  'word': 'google',
  'start': 0,
  'end': 6},
 {'entity_group': 'DATE',
  'score': 0.7211298,
  'word': '##bochi',
  'start': 16,
  'end': 21},
 {'entity_group': 'PER',
  'score': 0.9999806,
  'word': 'keshi',
  'start': 28,
  'end': 33},
 {'entity_group': 'PER',
  'score': 0.9999933,
  'word': 'obasanjo',
  'start': 39,
  'end': 47},
 {'entity_group': 'LOC',
  'score': 0.99997395,
  'word': 'naijiria',
  'start': 54,
  'end': 62},
 {'entity_group': 'DATE',
  'score': 0.99993277,
  'word': 'afo asato',
  'start': 63,
  'end': 72},
 {'entity_group': 'LOC',
  'score': 0.9999755,
  'word': 'legoosi steeti',
  'start': 132,
  'end': 146},
 {'entity_group': 'PER',
  'score': 0.99999523,
  'word': 'rasak fadipe',
  'start': 150,
  'end': 162},
 {'entity_group': 'DATE',
  'score': 0.99994236,
  'word': "n ' ehihie ubochi 24 jenuwari, 2018",
  'start': 213,
  'end': 246}]

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import shutil
shutil.move('/content/distilbert-base-uncased-nerR/checkpoint-5600','/content/gdrive/MyDrive/IBO_BETA/ LREC FINAL TRAINING/DistBert')

'/content/gdrive/MyDrive/IBO_BETA/ LREC FINAL TRAINING/DistBert'